In [1]:
# install libraries
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

In [3]:
# prompt: mount my google drive and get the mp3 file path in my drive

import os

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,

    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Device set to use cpu


README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

(…)-00000-of-00001-913508124a40cb97.parquet:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [5]:
# Replace 'path/to/your/mp3/file.mp3' with the actual path to your MP3 file in Google Drive
audio_file_path = "/content/drive/MyDrive/wisperAi/Asif_Mohiuddin.mp3"

# Check if the file exists
if os.path.exists(audio_file_path):
    print(f"MP3 file found at: {audio_file_path}")
else:
    print(f"Error: MP3 file not found at: {audio_file_path}")
    print("Please ensure the file exists and the path is correct.")

MP3 file found at: /content/drive/MyDrive/wisperAi/Asif_Mohiuddin.mp3


In [6]:
result = pipe(audio_file_path, return_timestamps=True)
print(f"Text: {result['text']}")


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Text:  আমি আস্তিপ মহয়দিন আমি ধাকা বাগনাদেশে জন্মেছি আমি উনিশের চুরাশী সালের চোব্ভিশে ফেব্রেরিতে জন্মেছি আমি একজন সেকুলের এক্টি঵িস্ট এবং একজন মানোবতাবাদি লেখোক বলগার শাঙ্বাদিক আমি মানোবতাবাদের পখ্যে লিখি সকলের সোমানধিকারের পখে লিখি, নারির সোমানধিকারের পখে, সমকামিদের সোমানধিকারের পখে, আস্তিক নাস্তিক সকলের সোমানধিকারের পখে
